In [21]:
import numpy as np
import scipy

In [172]:
n_assests = 10
A = np.random.randn(n_assests, n_assests)
covariance_matrix = np.dot(A, A.transpose())
expected_returns = np.random.randn(n_assests)

In [201]:
inv_cov = scipy.linalg.inv(covariance_matrix)
ones = np.ones((len(expected_returns), 1))

gmvp_w1 = (inv_cov @ ones) / (ones.transpose() @ inv_cov @ ones)
gmvp_var1 = gmvp_w1.transpose() @ covariance_matrix @ gmvp_w1
gmvp_std1 = np.sqrt(gmvp_var1)
gmvp_mu1 = gmvp_w1.transpose() @ expected_returns

gmvp_w1

array([[ 0.1223225 ],
       [ 0.11346039],
       [ 0.29431085],
       [ 0.30291061],
       [ 0.0820345 ],
       [-0.07932435],
       [ 0.3586077 ],
       [ 0.12011842],
       [-0.31880751],
       [ 0.00436687]])

In [263]:
n = len(expected_returns)  # (n,) shape

def variance(w):
    return w.T @ covariance_matrix @ w

# fun is a SciPy convention for objective functions, it is a 'equality' 'type' where it tries to equal to 0
constraints = [{
    'type': 'eq',
    'fun': lambda x: np.sum(x) - 1
}]

# initial guess
w0 = np.ones(n) / n

result1 = scipy.optimize.minimize(variance, w0, constraints=constraints)

w1 = result1.x
mu1 = w1.T @ expected_returns
var1 = w1.T @ covariance_matrix @ w1
std1 = np.sqrt(var1)

mu1

0.5418899231200734

In [259]:

bounds = [(0.0, 1.0) for _ in range(n)]

result2 = scipy.optimize.minimize(variance, w0, constraints=constraints, bounds=bounds)

w2 = result2.x
mu2 = w2.T @ expected_returns
var2 = w2.T @ covariance_matrix @ w2
std2 = np.sqrt(var2)

mu2

0.037558009901985726

In [256]:
np.sum(w2 <= 0.0)

1

In [258]:
bounds = [(0.05, 1.0) for _ in range(n)]

result3 = scipy.optimize.minimize(variance, w0, constraints=constraints, bounds=bounds)

w3 = result3.x
mu3 = w3.T @ expected_returns
var3 = w3.T @ covariance_matrix @ w3
std3 = np.sqrt(var3)

mu3

0.12974242626391141

In [291]:
long_constraints = [{
    'type': 'ineq',
    'fun': lambda w, i=i: w[i] - 1.05} for i in range(n)
]

constraints += long_constraints

result3 = scipy.optimize.minimize(variance, w0, constraints=constraints)

w3 = result3.x
mu3 = w3.T @ expected_returns
var3 = w3.T @ covariance_matrix @ w3
std3 = np.sqrt(var3)

mu3

'Optimizaiton failed: ' + result3.message


'Optimizaiton failed: Positive directional derivative for linesearch'

In [ ]:
sum_limit_constraint = [{'type': 'ineq', 'fun': lambda w: 0.20 - (w[0] +w[1] + w[2])}]
con

In [266]:
constraints += sum_limit_constraint

result3 = scipy.optimize.minimize(variance, w0, constraints=constraints)

w3 = result3.x
mu3 = w3.T @ expected_returns
var3 = w3.T @ covariance_matrix @ w3
std3 = np.sqrt(var3)


In [271]:
def optimize_portfolio(covariance_matrix, expected_returns, extra_constraints=None, 
                       bounds=None, w_min=0.0, w_max=1.0):
    
    n = len(expected_returns)

    def porfolio_variance(w):
        return w.T @ covariance_matrix @ w
    
    # normalize weights
    constraints = [{
        'type': 'eq',
        'fun': lambda w: np.sum(w) - 1
    }]

    if extra_constraints is not None:
        constraints += extra_constraints

    if bounds is None:
        bounds = [(w_min, w_max) for _ in range(n)]

    w0 = np.ones(n) / n

    result = scipy.optimize.minimize(porfolio_variance, w0, bounds=bounds, constraints=constraints)

    if not result.success:
        raise ValueError("Optimization failed: " + result.message)
    
    weights = result.x
    mu = weights @ expected_returns
    sigma = np.sqrt(weights.T @ covariance_matrix @ weights)

    return weights, mu, sigma




In [276]:
bounds = [(0.05, 1.0) for _ in range(n)]

optimize_portfolio(covariance_matrix=covariance_matrix, 
                   expected_returns=expected_returns,
                   bounds=bounds)

(array([0.05      , 0.14641986, 0.18081622, 0.05      , 0.10097247,
        0.10506397, 0.19641179, 0.05      , 0.05      , 0.07031569]),
 0.12974242626391141,
 0.6043271761691169)

In [275]:
bounds = [(0.00, 1.0) for _ in range(n)]

optimize_portfolio(covariance_matrix=covariance_matrix, 
                   expected_returns=expected_returns, 
                   bounds=bounds)

(array([4.90192805e-02, 1.49523270e-01, 1.30225778e-01, 0.00000000e+00,
        1.56662452e-01, 2.04658783e-01, 1.45246410e-01, 5.37522843e-02,
        1.31703459e-16, 1.10911743e-01]),
 0.037558009901985726,
 0.5016737781891024)

In [284]:
constraints = [{
    'type': 'ineq',
    'fun': lambda w: 0.20 - (w[0] + w[1] + w[2])
}] 

In [285]:
optimize_portfolio(covariance_matrix=covariance_matrix, 
                   expected_returns=expected_returns, 
                   bounds=bounds, extra_constraints=constraints)

(array([0.05      , 0.08843777, 0.06156223, 0.05      , 0.21617111,
        0.156137  , 0.08955817, 0.05      , 0.05      , 0.18813372]),
 -0.17741938552625003,
 0.707649734222466)